In [1]:
%%capture
!pip install zarr
!pip install s3fs
!pip install matplotlib
!pip install shapely
!pip install numcodecs


In [2]:
import matplotlib.pylab as plt
import numpy as np
import s3fs
import zarr
from numcodecs import LZMA, Blosc
from ipyleaflet import Map

We believe that AI should be a tool we can all just use now. And that it should be just 
easy-as to go grab some data and experiment/learn. Geophysical data doesn't always 
fit the bill, so we made this notebook. Enjoy!

The Ichthys3D is for the "2020 FORCE Machine Learning Contest"

https://xeek.ai/challenges/force-seismic/rules

This data for Ichthys field data is provided Geoscience Australia under CC-BY 4.0.

The "Ichthys 3D seismic for fault competition" segy data has been converted to ZArr, 
this notebook provides an example. 

In [3]:
# Location of the segy data in "ZArr format"
segy_path='s3://zarr-depot/seismic/Ichthys3D/Ichthys 3D seismic for fault competition'

In [4]:
# complicated stuff to access the data
client = s3fs.S3FileSystem(anon=True)
store = s3fs.S3Map(
    root=segy_path,
    s3=client, 
    check=False)
cache = zarr.LRUStoreCache(store, max_size=2**28)
# contains the seqy data for Ichthys3D
segy_data = zarr.open_group(store=cache, mode='r')

In [5]:
# contents:
[i for i in segy_data.keys()]

['coordinates',
 'indptr',
 'inline',
 'inlines',
 'map_inline_ends',
 'map_latitude',
 'map_longitude',
 'map_xline_ends',
 'trace_inlines',
 'trace_xlines',
 'xline',
 'xlines']

In [6]:
print ("inline range : ", segy_data['inlines'][:].min(), segy_data['inlines'][:].max())
print ("xline range : ", segy_data['xlines'][:].min(), segy_data['xlines'][:].max())

inline range :  1001 3400
xline range :  2040 3040


In [ ]:
def clip(img):
    """Standardize contrast"""
    img = img - np.mean(img)
    return img/np.std(img)

# These are 8-bit unsigned ints
plt.figure(figsize=(12,6))
plt.imshow(clip(segy_data['xline'][...,2222].T), cmap='gray', vmin=-2, vmax=2)
plt.show()

# slice data in the xline/inline direction
# Dead traces padded with 0's
plt.figure(figsize=(12,6))
plt.imshow(clip(segy_data['inline'][...,1499].T), cmap='gray', vmin=-2, vmax=2)
plt.show()

In [ ]:
%%timeit
x = segy_data['xline'][...,2400:2420]

### Geospatial data
Some geospatial data is included in the segy data. The cell below just renders a shapely polygon (latitude, longitude) bounding the survey. If you want to see a map, there is some 
more advanced configuration required. 

In [ ]:
from shapely.geometry import Polygon
# lat/lon bounding the survey
lati, longi = segy_data['map_latitude'][:], segy_data['map_longitude'][:]
points = list(zip(lati, longi))
Polygon(points)

### Advanced config

If you don't see a map appear, then you need some advanced config.
You'll only need to do this one time. 

Stop your notebook, and from the terminal, run these commands in order: 

**pip install ipyleaflet**

**jupyter nbextension enable --py --sys-prefix ipyleaflet**

**jupyter notebook**

This will add dependencies and restart the notebook with the ability to 
render the map below the cell following.

If everything worked you should see the top-end of 'Straya, and a red box which is
the survey.


In [ ]:
# for this to run, install ipyleaflet on command-line (see above)
from ipyleaflet import Map, Marker, Polygon, basemaps, basemap_to_tiles

m = Map(
    basemap=basemaps.Esri.WorldImagery,
    center=np.median(points, axis=0).tolist(),
    zoom=4
)

polygon = Polygon(
    locations=points,
    color="red",
    fill_color="green"
)

#m.add_layer(Marker(location=(52.204793, 360.121558)))
m.add_layer(polygon)
m